In [ ]:
import keras
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.io import wavfile

from sklearn.metrics import accuracy_score

from bokeh.plotting import figure, show
from bokeh.io import output_notebook


In [ ]:
from keras.layers import Conv1D, MaxPool1D, Activation, Dense, Input, Flatten
from keras.losses import binary_crossentropy
from keras.utils import Sequence
import tensorflow as tf

# Prepare Training set

In [ ]:
annotations_path = '../data/MagnaTagATune/annotation_reduced.csv'
annotations = pd.read_csv(annotations_path, sep='\t')
train_set, test_set = train_test_split(annotations['mp3_path'], train_size=0.01, test_size=0.1) 

x_train = []
x_test = []
for value in tqdm(train_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_train.append(data)
x_train = np.array(x_train)[:,:,np.newaxis]

for value in tqdm(test_set):
    path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
    _, data = wavfile.read(path)
    x_test.append(data)
x_test = np.array(x_test)[:,:,np.newaxis]

with open('x_train','wb') as outfile:
    pickle.dump(x_train,outfile)
with open('x_test','wb') as outfile:
    pickle.dump(x_test,outfile)

print("X train shape:",x_train.shape)
print("X test shape:",x_test.shape)

In [ ]:
train_set_paths = train_set.values
train_set_labels = annotations.loc[annotations['mp3_path'].isin(train_set)].drop(columns=['mp3_path','Unnamed: 0']).values
train_set_size = len(train_set_paths)
print("Train set size: {} ".format(train_set_size))

y_dimension = train_set_labels.shape[1]

_, data = wavfile.read( '../data/MagnaTagATune/rawwav/' + annotations['mp3_path'][0][:-3]+ 'wav')
x_dimension = len(data)

print("X dimension: {}\nY dimension: {}".format(x_dimension, y_dimension))

In [ ]:
class MagnaTagATuneSequence(Sequence):

    def __init__(self, train_set_paths, train_set_labels, batch_size):
        self.paths, self.y = train_set_paths, train_set_labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x_paths = self.paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = []
        for value in batch_x_paths:
            path = '../data/MagnaTagATune/rawwav/'+value[:-3]+'wav'
            _, data = wavfile.read(path)
            batch_x.append(data)
        batch_x = np.array(batch_x)[:,:,np.newaxis]
        return (batch_x,batch_y)

In [ ]:
config = tf.ConfigProto(device_count={'GPU':0})
#config.gpu_options.allow_growth = True
session = tf.Session(config=config)
keras.backend.set_session(session)

model = keras.Sequential()
model.add(Conv1D(filters=10, kernel_size=13, strides=6, input_shape=(x_dimension,1)))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size=3)) #
model.add(Conv1D(filters=4, kernel_size=3, strides=1))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=y_dimension, activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
callbacks = [keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True,
                                         write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, 
                                         embeddings_metadata=None)]
batch_size = 10
model.fit_generator(MagnaTagATuneSequence(train_set_paths, train_set_labels, batch_size),
                    epochs=2, callbacks = callbacks)
#model.fit(x_train,y_train,epochs=10, batch_size=32, callbacks=callbacks)

In [ ]:
predictions = model.predict(x_train)

In [ ]:
annotations.loc[annotations['mp3_path'].isin(test_set)].drop(columns=['mp3_path','Unnamed: 0']).columns.values[predictions[0] != 0]

In [ ]:
accuracy_score(y_train, predictions)